# Setup

## Google Only

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append('/content/drive/My Drive/Code/autocomplete_me')

In [5]:
!ls -l '/content/drive/My Drive/Code/autocomplete_me/src'

total 20
-rw------- 1 root root 2516 Jul 12 14:10 predict_utils.py
drwx------ 2 root root 4096 Jul 11 13:20 __pycache__
-rw------- 1 root root 3266 Jul 12 16:08 reader.py
-rw------- 1 root root 9341 Jul 12 14:24 utils.py


In [6]:
from src import utils, reader, predict_utils

Using TensorFlow backend.


In [7]:
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)

<module 'src.predict_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py'>

# Build Model

## Training

In [9]:
text = reader.read_bbc('entertainment')
content_type = 'BBC-Entertainment'

In [10]:
text[0]

'Blair buys copies of new Band Aid\n\nPrime Minister Tony Blair purchased two copies of the charity single Band Aid 20 in Edinburgh on Friday.\n\nStaff were surprised when the Prime Minister walked into HMV at 0900 GMT, accompanied by aides and local police. "When Mr Blair came in unannounced, we were all pretty gobsmacked," said HMV manager Clive Smith. "Our customer helper approached him... it was only then we realised he wanted to buy copies of the Band Aid single, rather than the latest Eminem album." Predicted chart-topper Do They Know it\'s Christmas? is expected to sell at least 300,000 copies by the time the new chart is announced on Sunday. However, the new version of the 1984 single is not going to be released in the US, despite being sold in many countries around the world. US record shops are stocking an import version of Do They Know It\'s Christmas, which is said to be selling very well in Los Angeles and New York. The original track was released in the US, and reached nu

In [ ]:
len(text)

### Short Text

In [11]:
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=20, short=True)

There are 15195 unique words.
There are 131676 sequences.


In [12]:
len(training_dict)

4

In [13]:
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 7098 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:180: RuntimeWarning: invalid value encountered in true_divide
  embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.06223089,  0.03835242,  0.08488411, ..., -0.04284497,
         0.08662399, -0.00527513],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05417296, -0.04813922,  0.13245429, ..., -0.20398643,
         0.01218687,  0.03430193]])

#### Unidirectional, 1 layer, Trainable

In [14]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False
)

In [15]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-20_seq',
    # model=model,
    use_pretrained_model=True,
    epochs=250
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 92173 samples, validate on 39503 samples
Epoch 1/250
92173/92173 [==============================] - 13s 146us/step - loss: 8.3708 - accuracy: 0.0279 - val_loss: 7.2150 - val_accuracy: 0.0464
Epoch 2/250
92173/92173 [==============================] - 11s 123us/step - loss: 7.1469 - accuracy: 0.0460 - val_loss: 7.1921 - val_accuracy: 0.0464
Epoch 3/250
92173/92173 [==============================] - 12s 125us/step - loss: 7.0660 - accuracy: 0.0455 - val_loss: 7.1606 - val_accuracy: 0.0464
Epoch 4/250
92173/92173 [==============================] - 11s 124us/step - loss: 6.9808 - accuracy: 0.0444 - val_loss: 7.1348 - val_accuracy: 0.0460
Epoch 5/250
92173/92173 [==============================] - 11s 124us/step - loss: 6.9245 - accuracy: 0.0458 - val_loss: 7.1498 - val_accuracy: 0.0464
Epoch 6/250
92173/92173 [==============================] - 11s 122us/step - loss: 6.8928 - accuracy: 0.0483 - val_loss: 7.1710 - val_accuracy: 0.0417
Epoch 7/250
92173/92173 [=========================

## Compare Model

<tr>
    <th>Name of Model</th>
    <th>LTSM Layers</th>
    <th>Bidirectional</th>
    <th>Trainable Embeddings</th>
    <th>Sequence Length</th>
    <th>LSTM Dropout</th>
    <th>LSTM Recurrent Dropout</th>
    <th>Val Loss</th>
    <th>Val Accuracy</th>
</tr>
<!-- <tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.5235</td>
    <td>0.1715</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.4229</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.2247</td>
    <td>0.1638</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.4449</td>
    <td>0.1725</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.2899</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.1667</td>
    <td>0.1719</td>
</tr> -->

# Sample Output

## Load Model

In [16]:
from keras.models import load_model
# Get Model Weights and Architecture

# MODELS_DIR = os.path.join(os.path.dirname(os.path.abspath('')), 'models')
MODELS_DIR = '/content/drive/My Drive/Code/autocomplete_me/models'
model_filepath = os.path.join(MODELS_DIR, f'{content_type}_uni-1_layer-trainable-20_seq.h5')

model = load_model(model_filepath)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [17]:
# Get Text Data
TRAINING_LENGTH = 20
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=TRAINING_LENGTH)

There are 15195 unique words.
There are 131676 sequences.


In [18]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=50,
    diversity=1,
    n_gen=1
)

In [19]:
' '.join(word for word in original_sequence)

'Coronation Street pulled away from its BBC One rival EastEnders in the ratings and dominated other TV awards . Last'

In [20]:
' '.join(word for word in gen_list[0])

'< --- > released of books to a respect to Fox by reveal of musical more turning each tells the one . Animation was month . somebody Of show . Christmas and to jail original news ago and him on money of the Shakespeare , crime Zach angry . rapper describing at The'

In [23]:
' '.join(word for word in a)

"< --- > year , the South Bank Award for best TV drama went to Steven Poliakoff's period piece The Lost Prince , while Bloody Sunday , about the 1972 killings in Northern Ireland , won in 2003. In other South Bank categories , Little Britain's second series beat Nighty Night and The"

In [24]:
sentence = 'Stocks of major large technology firms are becoming'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 5, 498, 1619, 3683, 13749, 33, 1141]
which after The Podium to been Christmas 1999 to and (Twentieth as seen a Other which of I but 13 rights refused and attention is date he this sing it's John McEwan the Their Trapp , . half in one Festival , week the artists at best in Mike incur


In [25]:
sentence = 'However, there have been many instances of'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 100, 27, 38, 189, None, 5]
Pictures wanting the whose ceremony Caroline broken of loopholes , in second also that 25th Fightstar that - May Taylor into garnered overcome ever deals on stage Live gallery prize it all the more station any any , well a £100,000 grail . Wal-Mart season of composer week and other
